In [37]:
import pandas as pd
df = pd.read_csv('../../customer.csv')
#for col in ('Review_Type', 'Rating_Value', 'Rating_Location', 'Rating_Sleep_Quality' , 'Rating_Rooms' ,'Rating_Cleanliness' ,'Rating_Service'):
df = df[df["Review_Type"].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [38]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [39]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [40]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [41]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_hotels",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/3d/_ydbxm491_7fx4q3z8lsrmlh0000gn/T/ipykernel_13489/2108492005.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [42]:
# vectorize!
qdrant.upload_points(
    collection_name="top_hotels",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Review_Type"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [43]:
user_prompt = "Suggest me an amazing hotel from Chandigarh for couple"

In [44]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_hotels",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'Hotel_Name_City': 'Hotel Shreenath Palace Jaisalmer', 'Review_Overall_Rating': 5, 'date_of_review': '8/20/2014', 'Month_of_Visit': 'Aug-14', 'Review_Type': 'as a couple', 'Rating_Value': 5.0, 'Rating_Location': 5.0, 'Rating_Sleep_Quality': nan, 'Rating_Rooms': nan, 'Rating_Cleanliness': nan, 'Rating_Service': 5.0} score: 0.17969600543227246
{'Hotel_Name_City': 'Hotel Classic Chandigarh Chandigarh', 'Review_Overall_Rating': 3, 'date_of_review': '6/28/2014', 'Month_of_Visit': 'Jul-13', 'Review_Type': 'as a couple', 'Rating_Value': 4.0, 'Rating_Location': 3.0, 'Rating_Sleep_Quality': 4.0, 'Rating_Rooms': 4.0, 'Rating_Cleanliness': 4.0, 'Rating_Service': 3.0} score: 0.17969600543227246
{'Hotel_Name_City': 'Mahua Bagh Resort Kashid', 'Review_Overall_Rating': 5, 'date_of_review': '3/25/2014', 'Month_of_Visit': 'Mar-14', 'Review_Type': 'as a couple', 'Rating_Value': 5.0, 'Rating_Location': 5.0, 'Rating_Sleep_Quality': 4.0, 'Rating_Rooms': 5.0, 'Rating_Cleanliness': 5.0, 'Rating_Service': 5.

/var/folders/3d/_ydbxm491_7fx4q3z8lsrmlh0000gn/T/ipykernel_13489/150458509.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [45]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]
print(search_results)

[{'Hotel_Name_City': 'Hotel Shreenath Palace Jaisalmer', 'Review_Overall_Rating': 5, 'date_of_review': '8/20/2014', 'Month_of_Visit': 'Aug-14', 'Review_Type': 'as a couple', 'Rating_Value': 5.0, 'Rating_Location': 5.0, 'Rating_Sleep_Quality': nan, 'Rating_Rooms': nan, 'Rating_Cleanliness': nan, 'Rating_Service': 5.0}, {'Hotel_Name_City': 'Hotel Classic Chandigarh Chandigarh', 'Review_Overall_Rating': 3, 'date_of_review': '6/28/2014', 'Month_of_Visit': 'Jul-13', 'Review_Type': 'as a couple', 'Rating_Value': 4.0, 'Rating_Location': 3.0, 'Rating_Sleep_Quality': 4.0, 'Rating_Rooms': 4.0, 'Rating_Cleanliness': 4.0, 'Rating_Service': 3.0}, {'Hotel_Name_City': 'Mahua Bagh Resort Kashid', 'Review_Overall_Rating': 5, 'date_of_review': '3/25/2014', 'Month_of_Visit': 'Mar-14', 'Review_Type': 'as a couple', 'Rating_Value': 5.0, 'Rating_Location': 5.0, 'Rating_Sleep_Quality': 4.0, 'Rating_Rooms': 5.0, 'Rating_Cleanliness': 5.0, 'Rating_Service': 5.0}]


In [46]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:11434/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="llama3.2:3b",
    messages=[
        {"role": "system", "content": "You are chatbot, a hotel specialist. Your top priority is to help guide users into selecting amazing hotel and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing hotel from Chandigarh for couple with top rating"},
        {"role": "assistant", "content": str(search_results)},
        {"role": "user", "content": "Based on that information, what would you recommend?"}
    ],
    max_tokens=500,  # Add max_tokens
    temperature=0.7  # Add temperature for more creative responses
)
#print(f"Full response: {completion}")
print(completion.choices[0].message.content)

Based on the provided information, I'd recommend the following hotels in Chandigarh for a couple's stay:

**Top Recommendation:** **Hotel Shreenath Palace Jaisalmer**

* Overall Rating: 5/5
* Review Type: As a Couple
* Room Quality: Excellent
* Cleanliness: Very Good
* Service: Exceptional

This hotel has received an excellent review from a couple, with all aspects of their stay rated as 5/5. It's located in Chandigarh and offers a romantic atmosphere, perfect for couples.

**Alternative Recommendation:** **Hotel Classic Chandigarh**

* Overall Rating: 4/5
* Review Type: As a Couple
* Room Quality: Good
* Cleanliness: Very Good
* Service: Good

This hotel has received a good review from a couple, with some areas for improvement. However, it still offers a comfortable stay and is located in the heart of Chandigarh.

**Third Option:** **Mahua Bagh Resort Kashid**

* Overall Rating: 5/5
* Review Type: As a Couple
* Room Quality: Excellent
* Cleanliness: Very Good
* Service: Exceptional

T

In [20]:
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:11434/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="llama3.2:3b",
    messages=[
        {"role": "system", "content": "You are a wine specialist chatbot. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)},
        {"role": "user", "content": "Based on that information, what would you recommend?"}
    ],
    max_tokens=500,
    temperature=0.7
)

print(completion.choices[0].message.content)

Considering the high ratings and reviews for these wines, I'd be happy to provide a recommendation.

For an amazing Malbec wine from Argentina, I'd suggest:

**Catena Zapata Nicasia Vineyard Malbec 2004**

This wine is rated 96.0 by critics and has been praised for its complex aroma profile (pain grille, scorched earth, mineral, licorice, blueberry, and black cherry) and thick, opulent palate with layers of fruit, silky tannins, and a long, fruit-filled finish.

Alternatively, if you prefer a more modern and affordable option:

**Altos las Hormigas Gualtallary Malbec 2017**

This wine is also highly rated (96.0) for its supple entry, dark berries, spices, firm fine-grain tannins, and thrilling freshness and balance.

Both wines are excellent choices, but if you're looking for a more classic and elegant style, the Catena Zapata Nicasia Vineyard Malbec 2004 might be the better fit. If you prefer a more modern and approachable take on Malbec, the Altos las Hormigas Gualtallary Malbec 2017